In [15]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import folium
import geopandas
import numpy as np
import os
import branca.colormap as cm
sns.set()

## Import data into Data Frame

We're using covidtracking.com for the data.  
Make a get request and convert the json response to a list.

In [3]:
stateData = requests.get('https://covidtracking.com/api/states/daily').json()

##  Put data into dataframe
Change dates from strings to datetime

In [4]:
# Put the data into a dataframe
stateDF = pd.DataFrame(stateData)
# Convert the date to a datetime
stateDF['date'] = pd.to_datetime(stateDF['date'].astype(str))


## Create a dataframe with most recent data (yesterday)

Using yesterday's data increases the likelihood that there will be data for all states (over using data for today which may still be coming in).


In [5]:
today = pd.to_datetime(datetime.date.today())
yesterday = today -  datetime.timedelta(days=1)
yesterdayDF = stateDF[stateDF['date'] == yesterday]

## Create a map color- coding yesterday's cases using Folium's Choropleth class
Doing it this way doesn't let us use data from the dataframe for tooltips.  But it's quick and easy.  

### GeoJSON file with state boundaries

In [8]:
stateGeoUrl = 'https://raw.githubusercontent.com/python-visualization/folium/master/examples/data/us-states.json'

# If you wanted to see what the info for one state looks like you could run the two lines below
# r = requests.get(stateGeoUrl).json()
# print(r['features'][1])


### Make the map

In [9]:
# Define our bins for color-coding based on quantiles of positive tests
bins = list(yesterdayDF['positive'].quantile([0, .25, .5, .75, 1]))

m = folium.Map(location=[48, -102], zoom_start=3)

positiveChoro = folium.Choropleth(
    name='Positive Tests',
    geo_data=stateGeoUrl,
    data=yesterdayDF,
    columns=['state', 'positive'],
    key_on='feature.id',
    fill_color='OrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Positive Tests',
    bins=bins,
    reset=True
)

#Add the Choropleth layer to the map
positiveChoro.add_to(m)

#Disply the map in Jupyter Lab by calling it's object
m

## Make a map putting everything into a geopandas dataframe and using Folium's GeoJSON class

This will give us a lot more customization options. 

First, make the geopandas dataframe by combining our state json and our covid data.  

In [11]:
geostate = geopandas.read_file(stateGeoUrl, driver='GeoJSON')

#index both dataframes on state abbreviations. 
geostate = geostate.set_index('id')
yesterdayDFindexed = yesterdayDF.set_index('state')

#An inner join will only keep rows where we have data in both sets
#This will automatically deal with things like Puerto Rico if it's in only one set
geostatedata = pd.concat([geostate, yesterdayDFindexed], axis=1, join='inner' )

#The JSON serialize run by folium to turn this into GeoJSON won't accept a datetime type
geostatedata['date'] = geostatedata['date'].astype(str)



Make a colormap for our color 'scale'


In [16]:
def makeColorMap(color1, color2, quantiles, column, highest_black=False):
    
    """
    Creates a color scale that has the same number of objects in each 
    color band even if the distribution is skewed.  If highest_black is True, 
    The last quantile will go to black. This is so we can better differentiate outliers.
    """
    
    bins = list(column.quantile(quantiles))
    colormap1= cm.LinearColormap(colors=[color1, color2], vmin=0, vmax=1)
    
    #colormap returns 8 character values but only accepts 6 characters
    colors = [colormap1(quantile)[0:-2] for quantile in quantiles]
    
    # change highest to black
    if highest_black: colors = colors[0:-1] + ['#000000']
    
    colormap = cm.LinearColormap(colors=colors, index=bins,
        vmin=column.min(),
        vmax=column.max())
    
    return colormap

In [23]:

quantiles = [0, 0.25, 0.5, 0.75, 0.98, 1]

#Make color map for positive tests
positiveColors = makeColorMap('orange', 'red', quantiles, geostatedata['positive'], highest_black=True)
positiveColors.caption = "Positive Covid Tests"

#Create a dictionay of colors because 'id' is the only property of the feature available when styling
colordict = geostatedata['positive'].apply(positiveColors)

#make color map for total tests
totalColors = makeColorMap('purple', 'blue', quantiles, geostatedata['total'], highest_black=True)
totalColors.caption = "Total Covid Tests"

#Create a dictionay of colors because 'id' is the only property of the feature available when styling
totalcolordict = geostatedata['total'].apply(totalColors)


Make colors for total tests

In [24]:

statemap = folium.Map(location=[48, -102], zoom_start=3)

statelayer = folium.GeoJson(
    geostatedata,
    name='States',
    style_function=lambda feature: {
        'fillColor': 'white',
        'fillOpacity': 0,
        'color': 'black',
        'weight': 1,
    },
    tooltip=folium.GeoJsonTooltip(
        fields=['name','positive', 'total'],
        aliases=['State','Positive Tests', 'Total Tests'],
        localize=True)
    )

positivelayer = folium.GeoJson(
    geostatedata,
    name='Positive Tests',
    style_function=lambda feature: {
        'fillColor':colordict[feature['id']],
        'fillOpacity': 0.5,
        'color': 'black',
        'weight': 1,
    }
    )

totallayer = folium.GeoJson(
    geostatedata,
    name='Total Tests',
    style_function=lambda feature: {
        'fillColor':totalcolordict[feature['id']],
        'fillOpacity': 0.5,
        'color': 'black',
        'weight': 1,
    }
    )


positivelayer.add_to(statemap)
totallayer.add_to(statemap)
statelayer.add_to(statemap)

statemap.add_child(positiveColors)
statemap.add_child(totalColors)

folium.LayerControl().add_to(statemap)

# #Save the map to an html file
# m.save(os.path.join('results', 'map.html'))

# #Disply the map in Jupyter Lab by calling it's object
statemap